<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-of-Python-modules" data-toc-modified-id="Import-of-Python-modules-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import of Python modules</a></span></li><li><span><a href="#Function-for-generation-of-2D-airfoil" data-toc-modified-id="Function-for-generation-of-2D-airfoil-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Function for generation of 2D airfoil</a></span></li></ul></div>

## Import of Python modules

In [190]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from scipy import interpolate
from mpl_toolkits.mplot3d import Axes3D

In [191]:
pathToFreeCAD = "F:/Progs/FreeCAD/FreeCAD-0.18.4.980bf90-WIN-x64-portable/conda-0.18.4/bin"
sys.path.append(pathToFreeCAD)

import FreeCAD, Part

## Function for generation of 2D airfoil

In [355]:
Beta_arr = np.array([15, 20, 25])
R_arr = np.array([35, 35, 100])
T_arr = np.array([2, 4, 3, 2])
    
def AxialCompressorAirfoil(L, Beta_array, R_array, T_array):
    FreeCAD.newDocument()
    N = 100                      # Number of the split average line of blade
    dx_i = np.linspace(0, L, N)    # Split of the horizontal length of the blade
    dx = dx_i[1:]-dx_i[:-1]
    dx_beta_func = np.linspace(0, L, Beta_array.shape[0])   # Split length of blade for generation function beta = f(x)
    beta_func = interpolate.interp1d(dx_beta_func, Beta_array, kind='quadratic') # function beta f(x)
    Beta_i = beta_func(dx_i[1:])       # Calculation of the beta values for dx coordinates    
    dy = dx*np.tan(Beta_i/180.0*np.pi)    # Calculation dy coordinates of the blade
    
    dy_i = np.array([dy[:i+1].sum() for i in range(dy.shape[0])])
    print(dy_i.shape[0])
    # Thickness of the streamline
    dx_thick_func = np.linspace(0, L, T_array.shape[0])
    thick_func = interpolate.interp1d(dx_thick_func, T_array, kind='quadratic')
    T_i = thick_func(dx_i[1:])
    
    dl = np.sqrt(dx**2+dy**2)
    Alpha_i = np.arctan(T_i/dl/2)
    dm = np.sqrt(dl**2+T_i**2)
    T_y1 = dm*np.sin(Alpha_i+(Beta_i/180*np.pi))
    T_x1 = dm*np.cos(Alpha_i+(Beta_i/180*np.pi))
    T_y2 = dm*np.sin((Beta_i/180*np.pi)-Alpha_i)
    T_x2 = dm*np.cos((Beta_i/180*np.pi)-Alpha_i)
    
    T_y1i = T_y1+dy_i
    T_x1i = T_x1+dx_i[1:]
    T_y2i = T_y2+dy_i
    T_x2i = T_x2+dx_i[1:]
    
    
    
    # Conformal mapping to the cylinder with user's Radius [mm]
    #x = R_array.min()*np.cos(dx/R_array.min())
    #y = R_array.min()*np.sin(dx/R_array.min())
    #z = dy
    
    # Conformal mapping to the surface with user's function R = f(dy)
    dx_merid_func = np.linspace(0, L, R_array.shape[0])  
    merid_func = interpolate.interp1d(dx_merid_func, R_array, kind='quadratic')
    R_i = merid_func(dx)
    x = R_i*np.cos(dx/R_i)
    y = R_i*np.sin(dx/R_i)
    z = dy
    Theta_array = 180/np.pi*(dx/R_i)
    
    
    
    
    
    
    
    
    
    return T_x1i, T_y1i, T_x2i, T_y2i

In [356]:
X, Y, Xs, Ys = AxialCompressorAirfoil(300, Beta_arr, R_arr, T_arr)

99


In [357]:
plt.plot(X, Y)
plt.plot(Xs, Ys)
plt.grid()

In [153]:
%matplotlib
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X, Y, Z)
ax.set_aspect('equal')
#ax = fig.gca(projection='3d')




# Create cubic bounding box to simulate equal aspect ratio
max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), Z.max()-Z.min()]).max()
Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(X.max()+X.min())
Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(Y.max()+Y.min())
Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(Z.max()+Z.min())
# Comment or uncomment following both lines to test the fake bounding box:
for xb, yb, zb in zip(Xb, Yb, Zb):
   ax.plot([xb], [yb], [zb], 'w')

Using matplotlib backend: Qt5Agg
